<a href="https://colab.research.google.com/github/mjiii25/posco-academy/blob/main/Big-Data/Assignment-2/vitalfew-DT-RF-GB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/mydrive')

Mounted at /content/mydrive


In [28]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report

### **데이터 구성하기**

In [73]:
df_raw = pd.read_excel('/content/mydrive/MyDrive/1. 실습데이터 설명(Data Dictionary).xlsx',
                       sheet_name = 'SCALE불량')

print(df_raw.shape)
df_raw.head()

(720, 21)


,PLATE_NO,ROLLING_DATE,SCALE,SPEC,STEEL_KIND,PT_THK,PT_WDTH,PT_LTH,PT_WGT,FUR_NO,...,FUR_HZ_TEMP,FUR_HZ_TIME,FUR_SZ_TEMP,FUR_SZ_TIME,FUR_TIME,FUR_EXTEMP,ROLLING_TEMP_T5,HSB,ROLLING_DESCALING,WORK_GR
0,PB562774,2008-08-01:00:00:15,양품,AB/EH32-TM,T1,32.25,3707,15109,14180,1호기,...,1144,116,1133,59,282,1133,934,적용,8,2조
1,PB562775,2008-08-01:00:00:16,양품,AB/EH32-TM,T1,32.25,3707,15109,14180,1호기,...,1144,122,1135,53,283,1135,937,적용,8,2조
2,PB562776,2008-08-01:00:00:59,양품,NV-E36-TM,T8,33.27,3619,19181,18130,2호기,...,1129,116,1121,55,282,1121,889,적용,8,3조
3,PB562777,2008-08-01:00:01:24,양품,NV-E36-TM,T8,33.27,3619,19181,18130,2호기,...,1152,125,1127,68,316,1127,885,적용,8,3조
4,PB562778,2008-08-01:00:01:44,양품,BV-EH36-TM,T8,38.33,3098,13334,12430,3호기,...,1140,134,1128,48,314,1128,873,적용,8,1조


In [74]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PLATE_NO           720 non-null    object 
 1   ROLLING_DATE       720 non-null    object 
 2   SCALE              720 non-null    object 
 3   SPEC               720 non-null    object 
 4   STEEL_KIND         720 non-null    object 
 5   PT_THK             720 non-null    float64
 6   PT_WDTH            720 non-null    int64  
 7   PT_LTH             720 non-null    int64  
 8   PT_WGT             720 non-null    int64  
 9   FUR_NO             720 non-null    object 
 10  FUR_NO_ROW         720 non-null    int64  
 11  FUR_HZ_TEMP        720 non-null    int64  
 12  FUR_HZ_TIME        720 non-null    int64  
 13  FUR_SZ_TEMP        720 non-null    int64  
 14  FUR_SZ_TIME        720 non-null    int64  
 15  FUR_TIME           720 non-null    int64  
 16  FUR_EXTEMP         720 non

**범주형 변수의 범주 확인하기**

In [75]:
df_raw['SPEC'].value_counts()

JS-SM490YB    84
LR-A          54
NV-A          48
PILAC-BT33    40
BV-EH36-TM    35
              ..
KR-B           1
BV-A           1
CCS-B          1
KS-HSB600      1
BV-AH36-TM     1
Name: SPEC, Length: 66, dtype: int64

In [76]:
df_raw['STEEL_KIND'].value_counts()

C0    503
T8     95
T5     43
T7     35
T1     18
T0     16
C3      7
T3      2
C1      1
Name: STEEL_KIND, dtype: int64

In [77]:
df_raw['FUR_NO'].value_counts()

3호기    243
1호기    240
2호기    237
Name: FUR_NO, dtype: int64

In [78]:
df_raw['FUR_NO_ROW'].value_counts()

1    362
2    358
Name: FUR_NO_ROW, dtype: int64

In [79]:
df_raw['FUR_NO_ROW'] = df_raw['FUR_NO_ROW'].astype('object')

In [80]:
df_raw['HSB'].value_counts()

적용     687
미적용     33
Name: HSB, dtype: int64

In [81]:
df_raw['WORK_GR'].value_counts()

4조    194
1조    189
3조    172
2조    165
Name: WORK_GR, dtype: int64

**결측값 확인하기**

In [68]:
df_raw_dummy.isnull().sum()

PT_THK               0
PT_WDTH              0
PT_LTH               0
PT_WGT               0
FUR_HZ_TEMP          0
FUR_HZ_TIME          0
FUR_SZ_TEMP          0
FUR_SZ_TIME          0
FUR_TIME             0
FUR_EXTEMP           0
ROLLING_TEMP_T5      0
ROLLING_DESCALING    0
SCALE_불량             0
SCALE_양품             0
STEEL_KIND_C0        0
STEEL_KIND_C1        0
STEEL_KIND_C3        0
STEEL_KIND_T0        0
STEEL_KIND_T1        0
STEEL_KIND_T3        0
STEEL_KIND_T5        0
STEEL_KIND_T7        0
STEEL_KIND_T8        0
FUR_NO_1호기           0
FUR_NO_2호기           0
FUR_NO_3호기           0
FUR_NO_ROW_1         0
FUR_NO_ROW_2         0
HSB_미적용              0
HSB_적용               0
WORK_GR_1조           0
WORK_GR_2조           0
WORK_GR_3조           0
WORK_GR_4조           0
dtype: int64

**목표변수와 설명변수 분리하기**

In [82]:
df_raw

,PLATE_NO,ROLLING_DATE,SCALE,SPEC,STEEL_KIND,PT_THK,PT_WDTH,PT_LTH,PT_WGT,FUR_NO,...,FUR_HZ_TEMP,FUR_HZ_TIME,FUR_SZ_TEMP,FUR_SZ_TIME,FUR_TIME,FUR_EXTEMP,ROLLING_TEMP_T5,HSB,ROLLING_DESCALING,WORK_GR
0,PB562774,2008-08-01:00:00:15,양품,AB/EH32-TM,T1,32.25,3707,15109,14180,1호기,...,1144,116,1133,59,282,1133,934,적용,8,2조
1,PB562775,2008-08-01:00:00:16,양품,AB/EH32-TM,T1,32.25,3707,15109,14180,1호기,...,1144,122,1135,53,283,1135,937,적용,8,2조
2,PB562776,2008-08-01:00:00:59,양품,NV-E36-TM,T8,33.27,3619,19181,18130,2호기,...,1129,116,1121,55,282,1121,889,적용,8,3조
3,PB562777,2008-08-01:00:01:24,양품,NV-E36-TM,T8,33.27,3619,19181,18130,2호기,...,1152,125,1127,68,316,1127,885,적용,8,3조
4,PB562778,2008-08-01:00:01:44,양품,BV-EH36-TM,T8,38.33,3098,13334,12430,3호기,...,1140,134,1128,48,314,1128,873,적용,8,1조
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,PB563502,2008-08-02:13:35:36,불량,NK-KA,C0,20.14,3580,38639,21870,3호기,...,1172,72,1164,62,245,1164,1005,적용,8,2조
716,PB563503,2008-08-02:13:35:02,양품,NV-A32,C0,15.08,3212,48233,18340,2호기,...,1150,61,1169,61,238,1169,947,적용,10,1조
717,PB563504,2008-08-02:14:40:00,양품,NV-A32,C0,16.60,3441,43688,19590,2호기,...,1169,65,1163,77,247,1163,948,적용,10,4조
718,PB563505,2008-08-02:13:35:19,양품,LR-A,C0,15.59,3363,48740,80240,3호기,...,1179,86,1163,45,243,1163,940,적용,10,2조


In [84]:
df_raw_x = df_raw.drop('SCALE', axis = 1, inplace = False)
df_raw_y = df_raw['SCALE']

In [85]:
df_raw_y.value_counts()

양품    489
불량    231
Name: SCALE, dtype: int64

**불필요한 설명변수 제외하기 & 더미변수 생성하기**

In [86]:
df_raw_x.columns

Index(['PLATE_NO', 'ROLLING_DATE', 'SPEC', 'STEEL_KIND', 'PT_THK', 'PT_WDTH',
       'PT_LTH', 'PT_WGT', 'FUR_NO', 'FUR_NO_ROW', 'FUR_HZ_TEMP',
       'FUR_HZ_TIME', 'FUR_SZ_TEMP', 'FUR_SZ_TIME', 'FUR_TIME', 'FUR_EXTEMP',
       'ROLLING_TEMP_T5', 'HSB', 'ROLLING_DESCALING', 'WORK_GR'],
      dtype='object')

In [87]:
df_raw_x = df_raw_x.drop(['PLATE_NO', 'ROLLING_DATE', 'SPEC'], axis = 1, inplace = False)

In [88]:
df_raw_x_dummy = pd.get_dummies(df_raw_x)

print(df_raw_x_dummy.shape)
df_raw_x_dummy.head()

(720, 32)


,PT_THK,PT_WDTH,PT_LTH,PT_WGT,FUR_HZ_TEMP,FUR_HZ_TIME,FUR_SZ_TEMP,FUR_SZ_TIME,FUR_TIME,FUR_EXTEMP,...,FUR_NO_2호기,FUR_NO_3호기,FUR_NO_ROW_1,FUR_NO_ROW_2,HSB_미적용,HSB_적용,WORK_GR_1조,WORK_GR_2조,WORK_GR_3조,WORK_GR_4조
0,32.25,3707,15109,14180,1144,116,1133,59,282,1133,...,0,0,1,0,0,1,0,1,0,0
1,32.25,3707,15109,14180,1144,122,1135,53,283,1135,...,0,0,0,1,0,1,0,1,0,0
2,33.27,3619,19181,18130,1129,116,1121,55,282,1121,...,1,0,1,0,0,1,0,0,1,0
3,33.27,3619,19181,18130,1152,125,1127,68,316,1127,...,1,0,0,1,0,1,0,0,1,0
4,38.33,3098,13334,12430,1140,134,1128,48,314,1128,...,0,1,1,0,0,1,1,0,0,0


In [89]:
df_raw_x_dummy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PT_THK             720 non-null    float64
 1   PT_WDTH            720 non-null    int64  
 2   PT_LTH             720 non-null    int64  
 3   PT_WGT             720 non-null    int64  
 4   FUR_HZ_TEMP        720 non-null    int64  
 5   FUR_HZ_TIME        720 non-null    int64  
 6   FUR_SZ_TEMP        720 non-null    int64  
 7   FUR_SZ_TIME        720 non-null    int64  
 8   FUR_TIME           720 non-null    int64  
 9   FUR_EXTEMP         720 non-null    int64  
 10  ROLLING_TEMP_T5    720 non-null    int64  
 11  ROLLING_DESCALING  720 non-null    int64  
 12  STEEL_KIND_C0      720 non-null    uint8  
 13  STEEL_KIND_C1      720 non-null    uint8  
 14  STEEL_KIND_C3      720 non-null    uint8  
 15  STEEL_KIND_T0      720 non-null    uint8  
 16  STEEL_KIND_T1      720 non

In [90]:
df_raw_x_dummy.describe().round(3)

,PT_THK,PT_WDTH,PT_LTH,PT_WGT,FUR_HZ_TEMP,FUR_HZ_TIME,FUR_SZ_TEMP,FUR_SZ_TIME,FUR_TIME,FUR_EXTEMP,...,FUR_NO_2호기,FUR_NO_3호기,FUR_NO_ROW_1,FUR_NO_ROW_2,HSB_미적용,HSB_적용,WORK_GR_1조,WORK_GR_2조,WORK_GR_3조,WORK_GR_4조
count,720.000,720.000,720.000,720.000,720.000,720.000,720.000,720.000,720.000,720.000,...,720.000,720.000,720.000,720.000,720.000,720.000,720.000,720.000,720.000,720.000
mean,29.268,2807.128,34323.269,41481.778,1155.339,88.528,1149.667,78.803,300.564,1149.667,...,0.329,0.338,0.503,0.497,0.046,0.954,0.262,0.229,0.239,0.269
std,19.153,517.840,13828.435,24751.322,20.866,37.661,17.572,36.421,45.390,17.572,...,0.470,0.473,0.500,0.500,0.209,0.209,0.440,0.421,0.427,0.444
min,12.030,1800.000,7893.000,7820.000,1103.000,43.000,1113.000,35.000,221.000,1113.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,16.090,2466.000,22663.000,17757.500,1139.000,65.000,1132.000,57.750,257.000,1132.000,...,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000
50%,20.130,2725.000,37380.000,36120.000,1158.000,75.000,1155.000,67.500,304.000,1155.000,...,0.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000
75%,40.000,3125.000,43735.000,61120.000,1171.000,101.000,1164.000,86.000,342.000,1164.000,...,1.000,1.000,1.000,1.000,0.000,1.000,1.000,0.000,0.000,1.000
max,100.340,4575.000,54917.000,115080.000,1206.000,312.000,1185.000,294.000,398.000,1185.000,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000


In [91]:
df_train_x, df_test_x, df_train_y, df_test_y = train_test_split(df_raw_x_dummy,
                                                                df_raw_y,
                                                                test_size = 0.3)

print('Shape of df_train_x : {}'.format(df_train_x.shape))
print('Shape of df_test_x : {}'.format(df_test_x.shape))
print('Shape of df_train_y : {}'.format(df_train_y.shape))
print('Shape of df_test_y : {}'.format(df_test_y.shape))

Shape of df_train_x : (504, 32)
Shape of df_test_x : (216, 32)
Shape of df_train_y : (504,)
Shape of df_test_y : (216,)


In [92]:
print(df_train_x.shape)
df_train_x.head()

(504, 32)


,PT_THK,PT_WDTH,PT_LTH,PT_WGT,FUR_HZ_TEMP,FUR_HZ_TIME,FUR_SZ_TEMP,FUR_SZ_TIME,FUR_TIME,FUR_EXTEMP,...,FUR_NO_2호기,FUR_NO_3호기,FUR_NO_ROW_1,FUR_NO_ROW_2,HSB_미적용,HSB_적용,WORK_GR_1조,WORK_GR_2조,WORK_GR_3조,WORK_GR_4조
136,20.10,2664,45986,57990,1159,95,1165,65,318,1165,...,1,0,0,1,0,1,1,0,0,0
678,19.93,3112,45576,44380,1170,72,1163,62,251,1163,...,0,1,1,0,0,1,0,1,0,0
394,18.62,3111,42179,38360,1195,72,1165,49,240,1165,...,0,0,1,0,0,1,0,0,0,1
103,14.09,3284,49142,89250,1133,69,1143,123,359,1143,...,0,0,0,1,0,1,1,0,0,0
239,22.66,2339,34345,42870,1133,89,1148,63,269,1148,...,0,1,0,1,0,1,0,0,0,1


In [93]:
print(df_train_y.shape)
df_train_y.head()

(504,)


136    불량
678    양품
394    양품
103    양품
239    불량
Name: SCALE, dtype: object

In [94]:
print(df_test_x.shape)
df_test_x.head()

(216, 32)


,PT_THK,PT_WDTH,PT_LTH,PT_WGT,FUR_HZ_TEMP,FUR_HZ_TIME,FUR_SZ_TEMP,FUR_SZ_TIME,FUR_TIME,FUR_EXTEMP,...,FUR_NO_2호기,FUR_NO_3호기,FUR_NO_ROW_1,FUR_NO_ROW_2,HSB_미적용,HSB_적용,WORK_GR_1조,WORK_GR_2조,WORK_GR_3조,WORK_GR_4조
679,20.14,2800,51753,22910,1180,63,1170,61,252,1170,...,0,1,0,1,0,1,0,1,0,0
561,22.66,2513,46374,82920,1163,229,1158,60,342,1158,...,0,1,1,0,0,1,0,0,0,1
168,20.10,2574,45600,92600,1165,78,1163,71,314,1163,...,0,0,1,0,0,1,0,0,0,1
246,16.09,2559,37250,36120,1153,67,1151,44,269,1151,...,0,0,1,0,1,0,0,0,1,0
138,20.10,2569,48822,98950,1162,111,1164,69,316,1164,...,0,0,0,1,0,1,0,1,0,0


In [95]:
print(df_test_y.shape)
df_test_y.head()

(216,)


679    양품
561    불량
168    불량
246    불량
138    불량
Name: SCALE, dtype: object

### **변수 중요도 확인하기**

#### **의사결정나무(Decision Tree)**

In [97]:
dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(df_train_x, df_train_y)

print('Accuracy on train set : {:.3f}'.format(dt_classifier.score(df_train_x, df_train_y)))
print('Accuracy on test set : {:.3f}'.format(dt_classifier.score(df_test_x, df_test_y)))

Accuracy on train set : 1.000
Accuracy on test set : 0.995


- train과 test 데이터에 대해 각각 100%, 99.5%의 설명력을 갖는다.
- train 데이터에 있어서 과대적합된 것으로 보인다.

In [98]:
dt_importance = pd.DataFrame()
dt_importance['Feature'] = df_train_x.columns
dt_importance['Importance'] = dt_classifier.feature_importances_

dt_importance.sort_values('Importance', ascending = False, inplace = True)
dt_importance.round(3).head()

,Feature,Importance
10,ROLLING_TEMP_T5,0.592
26,HSB_미적용,0.151
6,FUR_SZ_TEMP,0.094
0,PT_THK,0.086
11,ROLLING_DESCALING,0.063


**<모델 성능>**
- 기본 의사결정나무는 train과 test 데이터에 대해 각각 100%, 99.5%의 성능을 보여주고 있다. 
- Train 데이터에 있어서 과대적합인 것으로 확인된다.


**<변수 중요도>**
- 의사결정나무 모델을 통해 변수 중요도를 추출해보았다. 
- ROLLING_TEMP_T5가 0.592, HSB_미적용이 0.151의 변수 중요도를 보여주고 있다.

#### **랜덤 포레스트 (Random Forest)**

In [99]:
rf_classifier = RandomForestClassifier()
rf_classifier.fit(df_train_x, df_train_y)

print('Model Score on train set : {:.3f}'.format(rf_classifier.score(df_train_x, df_train_y)))
print('Model Score on test set : {:.3f}'.format(rf_classifier.score(df_test_x, df_test_y)))

Model Score on train set : 1.000
Model Score on test set : 0.968


In [100]:
rf_importance = pd.DataFrame()
rf_importance['Feature'] = df_train_x.columns
rf_importance['Importance'] = rf_classifier.feature_importances_

rf_importance.sort_values('Importance', ascending = False, inplace = True)
rf_importance.round(3).head()

,Feature,Importance
10,ROLLING_TEMP_T5,0.305
6,FUR_SZ_TEMP,0.081
1,PT_WDTH,0.076
9,FUR_EXTEMP,0.072
11,ROLLING_DESCALING,0.059


**<모델 성능>**
- 기본 랜덤포레스트 예측 모델은 train과 test 데이터에 대해 각각 100%, 96.8%의 설명력을 갖는다.
- 의사결정나무에 비해서는 과대적합의 경향성이 적어보이지만, 여전히 모델의 일반화 가능성은 높아 보이지 않는다.

**<변수 중요도>**
- ROLLING_TEMP_T5가 0.305, FUR_SZ_TEMP가 0.081의 변수 중요도를 가진다.

#### **그래디언트 부스팅 (Gradient Boosting)**

In [101]:
gb_classifier = GradientBoostingClassifier()
gb_classifier.fit(df_train_x, df_train_y)

print('Model Score on train set : {:.3f}'.format(gb_classifier.score(df_train_x, df_train_y)))
print('Model Score on test set : {:.3f}'.format(gb_classifier.score(df_test_x, df_test_y)))

Model Score on train set : 1.000
Model Score on test set : 1.000


In [102]:
gb_importance = pd.DataFrame()
gb_importance['Feature'] = df_train_x.columns
gb_importance['Importance'] = gb_classifier.feature_importances_

gb_importance.sort_values('Importance', ascending = False, inplace = True)
gb_importance.round(3).head()

,Feature,Importance
10,ROLLING_TEMP_T5,0.612
26,HSB_미적용,0.098
11,ROLLING_DESCALING,0.080
9,FUR_EXTEMP,0.074
27,HSB_적용,0.058


**<모델 성능>**
- 기본 그래디언트 부스팅 모델은 train과 test 데이터에 대해 각각 100%, 100%의 설명력을 갖는다.
- 의사결정나무에 비해 test 데이터에 대한 성능은 좋지만, 지나친 과대적합의 가능성이 우려된다.

**<변수 중요도>**
- ROLLING_TEMP_T5가 0.612, HSB_미적용이 0.098의 변수 중요도를 갖는다.

### **결론**

- 의사결정나무, 랜덤 포레스트, 그리고 그래디언트 부스팅 예측기를 이용하여 추출한 변수 중요도는 다음과 같다.


| 모델명/순위 |  의사결정나무  |  랜덤 포레스트   |  그래디언트 부스팅  |
|:----------:|:--------------:|:----------------:|:------------------:|
|1|ROLLING_TEMP_T5 (0.592)|ROLLING_TEMP_T5 (0.305)|ROLLING_TEMP_T5 (0.612)|
|2|HSB_미적용 (0.151)     |FUR_SZ_TEMP (0.081) | HSB_미적용 (0.098)         |
|3|FUR_SZ_TEMP (0.094)    |PT_WDTH (0.076)      |ROLLING_DESCALING (0.080)        |
|4|PT_THK (0.086)         |FUR_EXTEMP (0.072)   |FUR_EXTEMP (0.074)       |
|5|ROLLING_DESCALING (0.063)|ROLLING_DESCALING (0.059) |HSB_적용 (0.058)    |

* 3개의 예측 모델 모두 중요하게 고려한 변수는 ROLLING_TEMP_T5, ROLLING_DESCALING이다.
* 압연온도와 압연횟수가 불량제품 생산에 유의한 영향을 미친다고 볼 수 있다.